In [1]:
%reset -f

from functools import reduce
import sys
import time
import math
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
from datetime import datetime

from stats import Stats
from character import Character

# File path and name
file_path = "res.txt"
SLOT_NUMER = 37

#============getting all the items from the db.csv file=========================#
print(f'Fetching items from db: Started')
start_time = time.time()

all_items       = []
all_items_ids   = { k:[] for k in range(SLOT_NUMER)}

df = pd.read_csv('db.csv')
for index, row in df.iterrows():
    d = row.dropna().to_dict()
    
    all_items_ids[d['slot']].append(d['id'])
    all_items.append(Stats(d['slot'], d['name'], d['set'],attributes=d))
print(len(all_items), all_items)
#=======filter items_in_slot and prepare for the combination function===========#
all_items_ids_filtered = list(filter(None, all_items_ids.values()))

#================print some info about the objects loaded=======================#
for slot, item in all_items_ids.items():
    if len(item) > 0:
        print(f'slot: {slot:3}, choices:{len(item):3}')
all_items_ids_filtered_len = reduce(lambda a, b: a*len(b), [1,*all_items_ids_filtered])
print(f'Items combinations: {all_items_ids_filtered_len}')

print(f'Fetching items from db: Completed ({time.time()-start_time}s)')
print(f'Size all_items_ids: {sys.getsizeof(all_items_ids)}')
print(f'Size all_items: {sys.getsizeof(all_items)}')

Fetching items from db: Started
36 [<stats.Stats object at 0x00000179BF226B90>, <stats.Stats object at 0x00000179BF226710>, <stats.Stats object at 0x00000179BF226050>, <stats.Stats object at 0x00000179BF226650>, <stats.Stats object at 0x00000179BF2267A0>, <stats.Stats object at 0x00000179BF2268F0>, <stats.Stats object at 0x00000179BF226D70>, <stats.Stats object at 0x00000179BF226DD0>, <stats.Stats object at 0x00000179BF2268C0>, <stats.Stats object at 0x00000179BF226D40>, <stats.Stats object at 0x00000179BF2269B0>, <stats.Stats object at 0x00000179BF226A10>, <stats.Stats object at 0x00000179BF226A70>, <stats.Stats object at 0x00000179BF226AD0>, <stats.Stats object at 0x00000179BF226E60>, <stats.Stats object at 0x00000179BF226EC0>, <stats.Stats object at 0x00000179BF226F20>, <stats.Stats object at 0x00000179BF226F80>, <stats.Stats object at 0x00000179BF226FE0>, <stats.Stats object at 0x00000179BF227040>, <stats.Stats object at 0x00000179BF2270A0>, <stats.Stats object at 0x00000179BF22710

In [2]:
#==============base stats for a specific class (not present in db)==============#
base_class_stats    = { 'phy_atk':1202,	'mag_atk':1202,	'phy_def':301,	'mag_def':301,	'hp':157000,
                        'dmg_per':      18,  
                        'dmg_per_m':    0,
                        'crit':         25,        
                        'maxi':         0,        
                        'crit_dmg':     25,

                        'pola':         0,
                        'specific':     0,
                        'all_s_dmg':    3,
                        'boss_dmg':     0,
                        'adapt':        0,
                        'cdr':          0
                       }

#===============================force passives==================================#
head_hunter = {'dmg_per':-24, 'boss_dmg':80}


#============================sockets combinations===============================#
avail_sockets = [  
                    Stats('0', 'socket', attributes={'maxi':12}),
                    Stats('0', 'socket', attributes={'crit':12}),
                    Stats('0', 'socket', attributes={'boss_dmg':5})
                ]

print(f'Generating sockets combinations: Started')
start_time = time.time()
# generate combinations
sockets_combinations = list(combinations_with_replacement(avail_sockets, 2))

sockets_combinations_len = len(sockets_combinations)
print(f'Socket combinations: {sockets_combinations_len}')

# generate item for each socket combination, grouping sockets into a single item
itemized_sockets = []
for sockets in sockets_combinations:
    tot_socket = Stats('0', 'SOCKETS')
    for socket in sockets:
        tot_socket += socket
    itemized_sockets.append(tot_socket)

print(len(itemized_sockets))
itemized_sockets = list(filter(Stats.maxed_crit, itemized_sockets))
print(len(itemized_sockets))
itemized_sockets = list(filter(Stats.maxed_maxi, itemized_sockets))
print(len(itemized_sockets))

itemized_sockets_ids = np.arange(len(itemized_sockets))

print(f'Generating sockets combinations: Completed ({time.time()-start_time}s)')
print(f'Size itemized_sockets: {sys.getsizeof(itemized_sockets)}')

Generating sockets combinations: Started
Socket combinations: 6
6
6
6
Generating sockets combinations: Completed (0.0s)
Size itemized_sockets: 120


In [3]:
#=========================combinatory for all items=============================#
print('Generating Items combinations, Starting')
start_time = time.time()

combs = np.array(np.meshgrid(*all_items_ids_filtered, itemized_sockets_ids))
combs = combs.T.reshape(-1,len([*all_items_ids_filtered, itemized_sockets_ids]))
print(combs)

#combs = list(product(*list(items_in_slot_filtered), itemized_sockets))

#combs = random.sample(combs, 100000)
#combs = combs[::500]

combs_len = len(combs)
print(f'Total combinations: {combs_len}')

print(f'Total combinations as expected: {combs_len==(all_items_ids_filtered_len*sockets_combinations_len)}')
print(f'Combinations, Complete ({time.time()-start_time}s)')

print(f'Size combs: {sys.getsizeof(combs)}')



Generating Items combinations, Starting
[[21  0  1 ... 25 35  0]
 [21  4  1 ... 25 35  0]
 [21  8  1 ... 25 35  0]
 ...
 [31  4 13 ... 34 35  5]
 [31  8 13 ... 34 35  5]
 [31 12 13 ... 34 35  5]]
Total combinations: 589824
Total combinations as expected: True
Combinations, Complete (0.21065187454223633s)
Size combs: 128


In [4]:
from threading import Thread

def task(tid, combs, i, j, itemized_sockets, res):
    print(i,j)
    #=============================create specific items ============================#
    base_class_stats_item = Stats('base','class base stats','',attributes=base_class_stats)
    head_hunter_force_passive = Stats('HH','unique HH','',attributes=head_hunter)

    new_character = Character('Char', '')
    new_character = new_character.equip(base_class_stats_item, base_class_stats_item.slot)
    new_character = new_character.equip(head_hunter_force_passive, head_hunter_force_passive.slot)
    
    for ii, items in enumerate(combs[i:j]):
        # create a new "character" for each item combination
        new_character.name = 'jeff_%s_%s'%(tid, ii)
        new_character.items = {}
        new_character.attributes = {}

        # add the sockets item with is a Stats object
        
        new_character = new_character.equip(itemized_sockets[items[-1]], itemized_sockets[items[-1]].slot)
        
        # equip all itemsD
        for id in items[:-1]:
            new_character = new_character.equip(all_items[id], all_items[id].slot)
        
        res[tid*(j-i)+ii] = new_character.get_mul()
num_tasks = 8

indexes = np.arange(0, len(combs)+1, len(combs)/num_tasks, dtype=int)
print(indexes)

res = np.zeros(len(combs))
threads = [Thread(target=task, args=(id, combs, indexes[i], indexes[i+1], itemized_sockets, res)) 
           for id, i in enumerate(np.arange(num_tasks)) ]

print(threads)
print('Item placement and damage calculation, Starting')
# run the threads
for t in threads:
    t.start()
# wait for the threads to finish
print('Waiting for the thread...')

mul_list = []

for t in threads:   
    t.join()
print(f'\nItem placement and damage calculation, Complete ({time.time()-start_time}s)')
#print(f'Size mul_list: {sys.getsizeof(mul_list)}')

print(res)


[     0  73728 147456 221184 294912 368640 442368 516096 589824]
[<Thread(Thread-5 (task), initial)>, <Thread(Thread-6 (task), initial)>, <Thread(Thread-7 (task), initial)>, <Thread(Thread-8 (task), initial)>, <Thread(Thread-9 (task), initial)>, <Thread(Thread-10 (task), initial)>, <Thread(Thread-11 (task), initial)>, <Thread(Thread-12 (task), initial)>]
Item placement and damage calculation, Starting
0 73728
73728 147456
147456 221184
221184 294912
294912 368640
368640 442368
442368 516096
516096 589824
Waiting for the thread...

Item placement and damage calculation, Complete (124.62538766860962s)
[ 6.54999093  6.75349501  7.24468763 ... 11.13896021 11.13273436
 11.17851495]


In [5]:
#==============apply the damage calculation to each character===================#
print(type(res))
mul_list = res
m = max(mul_list)
print(m)
best_comb_index = np.where(mul_list == m)[0]
print(best_comb_index)
best_comb = combs[best_comb_index][0]

print('best_comb',best_comb)


best_character = Character('Char', '')
base_class_stats_item = Stats('base','class base stats','',attributes=base_class_stats)
head_hunter_force_passive = Stats('HH','unique HH','',attributes=head_hunter)

best_character = best_character.equip(base_class_stats_item, base_class_stats_item.slot)
best_character = best_character.equip(head_hunter_force_passive, head_hunter_force_passive.slot)

best_character = best_character.equip(itemized_sockets[best_comb[-1]], itemized_sockets[best_comb[-1]].slot)

for id in best_comb[:-1]:
    best_character = best_character.equip(all_items[id], all_items[id].slot)

#print(best_character)
best = best_character.get_damage()

print(best)




<class 'numpy.ndarray'>
14.136846490425862
[69627 69631 86011 86015]
best_comb [21 12 13 14 15 26 27 28 19 20 22 23 24 34 35  0]
(11.007070629865396, 60153.64099221439, <character.Character object at 0x00000179BC9605B0>)


In [6]:
#=========================save to file best N entries===========================#
with open(file_path, "w") as file:
    file.write(f'Latest update: {datetime.now()}\n')
    
    output_string = f"{math.trunc(best[0]):<10}{math.trunc(best[1]):<10}\n{best[2]}\n"
    
    file.write(output_string.replace('|0.0','|   '))
    
print('DONE')

DONE
